In [20]:
import os
import re
import aocd
import numpy as np
import pandas as pd
from aocd import submit
from tqdm import tqdm
from collections import Counter


data = """CBNBOKHVBONCPPBBCKVH

FK -> O
BK -> B
PB -> N
VS -> P
OF -> H
KP -> K
PS -> K
OV -> N
FO -> H
KN -> P
HF -> K
BV -> N
OO -> B
KC -> V
CK -> H
BC -> P
VV -> S
NS -> C
SF -> O
BN -> V
NH -> N
VP -> F
KH -> S
BO -> N
VN -> K
BB -> H
CH -> H
HP -> O
KK -> O
CB -> S
VC -> P
FH -> B
SP -> C
NF -> O
HN -> N
PO -> P
PP -> C
SO -> F
FB -> B
SB -> B
SC -> B
HK -> O
BF -> V
OB -> B
NC -> V
HC -> F
KO -> C
NV -> C
HB -> H
FP -> S
OS -> O
HH -> K
OK -> B
OH -> C
NP -> V
SN -> H
SK -> B
HV -> F
VF -> P
CP -> H
FN -> H
FV -> B
CN -> H
OC -> O
KV -> P
CF -> B
OP -> B
FC -> O
PC -> B
CV -> S
PV -> H
VK -> N
SS -> C
HO -> F
VH -> C
NB -> S
NN -> F
FF -> K
CC -> H
SV -> H
CO -> K
BP -> O
SH -> H
KS -> K
FS -> F
PF -> S
BS -> H
VO -> H
NK -> F
PK -> B
KB -> K
CS -> C
VB -> V
BH -> O
KF -> N
HS -> H
PH -> K
ON -> H
PN -> K
NO -> S""".split('\n\n')

test = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C""".split('\n\n')
#data = test

In [2]:
prev, rules = data
rules = rules.split('\n')
rules = [x.split(' -> ') for x in rules]
rules = {k:v for k,v in rules}
prev, rules
n_steps = 10
for i in tqdm(range(1,n_steps+1)):
    print(len(prev))
    counts = Counter(prev)
    print(counts)
    new = ''
    for j in range(len(prev)-1):
        pair = prev[j:j+2]
        if pair in rules.keys():
            new += prev[j] + rules[pair]
            
        if j == (len(prev)-2):
            new += prev[-1]
    prev = new
    #print(prev, new)

counts = Counter(prev)
counts

  0%|          | 0/40 [00:00<?, ?it/s]

4
Counter({'N': 2, 'C': 1, 'B': 1})
NCNBCHB NCNBCHB


Counter({'N': 2, 'C': 2, 'B': 2, 'H': 1})

In [21]:
from copy import deepcopy
#for each pair, we know which new pair will be created
prev, rules = data
rules = rules.split('\n')
rules = [x.split(' -> ') for x in rules]
#rules = {k:v for k,v in rules}

added_counts = []
for x,mid in rules:
    added_counts += [[x, [x[0]+mid,mid+x[1]]]]
added_counts = {k:v for k,v in added_counts}
added_counts, prev,Counter([prev[j:j+2] for j in range(len(prev)-1)])

({'FK': ['FO', 'OK'],
  'BK': ['BB', 'BK'],
  'PB': ['PN', 'NB'],
  'VS': ['VP', 'PS'],
  'OF': ['OH', 'HF'],
  'KP': ['KK', 'KP'],
  'PS': ['PK', 'KS'],
  'OV': ['ON', 'NV'],
  'FO': ['FH', 'HO'],
  'KN': ['KP', 'PN'],
  'HF': ['HK', 'KF'],
  'BV': ['BN', 'NV'],
  'OO': ['OB', 'BO'],
  'KC': ['KV', 'VC'],
  'CK': ['CH', 'HK'],
  'BC': ['BP', 'PC'],
  'VV': ['VS', 'SV'],
  'NS': ['NC', 'CS'],
  'SF': ['SO', 'OF'],
  'BN': ['BV', 'VN'],
  'NH': ['NN', 'NH'],
  'VP': ['VF', 'FP'],
  'KH': ['KS', 'SH'],
  'BO': ['BN', 'NO'],
  'VN': ['VK', 'KN'],
  'BB': ['BH', 'HB'],
  'CH': ['CH', 'HH'],
  'HP': ['HO', 'OP'],
  'KK': ['KO', 'OK'],
  'CB': ['CS', 'SB'],
  'VC': ['VP', 'PC'],
  'FH': ['FB', 'BH'],
  'SP': ['SC', 'CP'],
  'NF': ['NO', 'OF'],
  'HN': ['HN', 'NN'],
  'PO': ['PP', 'PO'],
  'PP': ['PC', 'CP'],
  'SO': ['SF', 'FO'],
  'FB': ['FB', 'BB'],
  'SB': ['SB', 'BB'],
  'SC': ['SB', 'BC'],
  'HK': ['HO', 'OK'],
  'BF': ['BV', 'VF'],
  'OB': ['OB', 'BB'],
  'NC': ['NV', 'VC'],
  'HC': ['

In [22]:
prev, rules
n_steps = 40
prev_counts = Counter([prev[j:j+2] for j in range(len(prev)-1)])
for i in tqdm(range(1,n_steps+1)):
    #print(i,prev_counts,'\n')
    new_counts = deepcopy(prev_counts)
    for c in prev_counts:
        #print('c',c)
        if prev_counts[c] == 0:
            #print('skipping, no values')
            continue
        additions = added_counts[c]
            
        for a in additions:
            #print('a',a)
            if new_counts.get(a,None) is None:
                new_counts[a] = 0
            new_counts[a] += prev_counts[c]
        new_counts[c] -= prev_counts[c]
    prev_counts = new_counts
pair_counts = new_counts

100%|██████████| 40/40 [00:00<00:00, 4437.95it/s]


In [31]:
indiv_counts = Counter()
for pair in pair_counts:
    #print(pair)
    for indiv in pair:
        if indiv_counts.get(indiv, None) is None:
            indiv_counts[indiv] = 0
        indiv_counts[indiv] += pair_counts[pair]
indiv_counts[prev[0]] += 1
indiv_counts[prev[-1]] += 1
for indiv in indiv_counts:
    indiv_counts[indiv] = indiv_counts[indiv]/2
indiv_counts

Counter({'C': 1428517996063.0,
         'B': 3703497099947.0,
         'N': 1127204489569.0,
         'O': 2921790654982.0,
         'K': 3428002915512.0,
         'H': 4439532321735.0,
         'V': 679219618858.0,
         'P': 750750900551.0,
         'S': 1162667583469.0,
         'F': 1249537347059.0})

In [32]:
4439532321735 - 679219618858

3760312702877

In [46]:
counts = Counter([prev[j:j+2] for j in range(len(prev)-1)])
counts['CB'] += 1
counts

Counter({'CB': 2,
         'BN': 1,
         'NB': 1,
         'BO': 2,
         'OK': 1,
         'KH': 1,
         'HV': 1,
         'VB': 1,
         'ON': 1,
         'NC': 1,
         'CP': 1,
         'PP': 1,
         'PB': 1,
         'BB': 1,
         'BC': 1,
         'CK': 1,
         'KV': 1,
         'VH': 1})

In [47]:
rules

for c in counts:
    print(c)

CB
BN
NB
BO
OK
KH
HV
VB
ON
NC
CP
PP
PB
BB
BC
CK
KV
VH
